In [1]:
#### MAY NEED TO REMOVE --> THESE IMPORTS ARE JUST FOR MY DEPENDENCIES ON MY LOCAL DEVICE
import sys 
sys.path.append('../..')
######

from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
import matplotlib.pyplot as plt
import seaborn as sns
import torch as ch
from torch import Tensor
from torch import sigmoid as sig
import torch.nn as nn
from torch.optim import SGD, lr_scheduler
from torch.distributions import Gumbel, Uniform
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.transforms import SigmoidTransform
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from tqdm.autonotebook import tqdm as tqdm
from abc import ABC
import IPython
import os
import dill
import config

from delphi import train
from delphi import oracle
from delphi import grad
import delphi.utils.constants as consts
# set environment variable so that stores can create output files

os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

if ch.cuda.is_available(): 
    ch.set_default_tensor_type(ch.cuda.FloatTensor)
    print("cuda tensors...")

cuda tensors...


In [2]:
config.args = Parameters({
    'in_features': 2, 
    'k': 2,
    'param_lower': -1, 
    'param_upper': 1, 
    'x_lower': -5, 
    'x_upper': 5,
    'samples': 10000,
    'num_samples': 1000,
})
config.args

{
  "in_features": 2,
  "k": 2,
  "param_lower": -1,
  "param_upper": 1,
  "x_lower": -5,
  "x_upper": 5,
  "samples": 10000,
  "num_samples": 1000
}

In [3]:
# oracles
id_ = oracle.Identity()

Distributions

In [4]:
gumbel = Gumbel(0, 1)

# logistic distribution
base_distribution = Uniform(0, 1)
transforms_ = [SigmoidTransform().inv]
logistic = TransformedDistribution(base_distribution, transforms_)

Gradients

In [5]:
logistic_bce = grad.LogisticBCE.apply
from torch.nn import BCEWithLogitsLoss; bce_logit = BCEWithLogitsLoss()
trunc_ce = grad.TruncatedCE.apply
trunc_bce = grad.TruncatedBCE.apply
gumbel_ce = grad.GumbelCE.apply
from torch.nn import CrossEntropyLoss; ce_loss = CrossEntropyLoss()

In [6]:
def gradient_similarity(args, grad_1, grad_2, d_n):
    """
    grad_1: first gradient
    grad_2: second gradient 
    d_n: latent variable noise distribution
    returns: grad_1 and grad_2 gradients with respect to the noised output
    """
    # generate random uniform weights
    ground_truth = nn.Linear(in_features=args.in_features, out_features=args.k, bias=args.bias)
    ground_truth.weight = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1, args.in_features])))
    if ground_truth.bias is not None: 
        ground_truth.bias = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1,])))

    # generate data
    X = Uniform(-5, 5).sample(ch.Size([args.samples, args.in_features]))
    z = ground_truth(X) + d_n.sample(ch.Size([X.size(0), 1]))
    y = args.pi(z)

    # truncate
    indices = ch.all(args.phi(z).bool(), dim=1).nonzero(as_tuple=False).flatten()
    x_trunc = X[indices]
    y_trunc = y[indices]

    print("alpha: ", x_trunc.size(0) / X.size(0))

    # pass data through model
    model = ch.nn.Linear(in_features=args.in_features, out_features=args.k)
    pred = model(x_trunc)

    g1_loss = grad_1(pred, y_trunc)
    g1_g, = ch.autograd.grad(g1_loss, [pred])

    g2_loss = grad_2(pred, y_trunc)
    g2_g, = ch.autograd.grad(g2_loss, [pred])


    print('Cosine Similarity between grad 1 and grad 2: ',
            ch.nn.CosineSimilarity(dim=1)(g2_g.T, g1_g.T))
    
    return g1_g, g2_g

Projection Mechanisms

In [7]:
binary = lambda z: ch.where(z > 0, ch.ones(1), ch.zeros(1))
multi = lambda z: z.argmax(-1)

Compare CE Loss with Truncated CE Loss with Identity Oracle

In [8]:
config.args.__setattr__('phi', id_)
config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
g1, g2 = gradient_similarity(config.args, ce_loss, trunc_ce, gumbel)

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([0.9990, 0.9806])


Compare CE Loss with Truncated CE Loss with Logit Ball Oracle

In [10]:
config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)) )
g1, g2 = gradient_similarity(config.args, ce_loss, trunc_ce, gumbel)

alpha:  0.5989
Cosine Similarity between grad 1 and grad 2:  tensor([0.9425, 0.4788])


Compare BCE Loss with Truncated BCE Loss and Idenity Oracle

In [11]:
config.args.__setattr__('phi', id_)
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce, logistic)

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([0.9998])


Compare BCE Loss with Truncated BCE Loss with Ball Oracle

In [12]:
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)) )
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce, logistic)

alpha:  0.4714
Cosine Similarity between grad 1 and grad 2:  tensor([0.9279])
